# SAT

SAT-Problem (Satisfiability Problem) is a problem of determining whether a given Boolean formula is satisfiable. A formula is satisfiable if all clauses in the formula are true. A clause is a disjunction of literals. A literal is a variable or its negation.

For example:

$(x_1 \lor x_2) \land (\neg x_1 \lor x_3) \land (\neg x_2 \lor \neg x_3)$

Components of the formula are as follows:
- $x_1, x_2, x_3$ are variables
- $\neg$ is the negation
- $\lor$ is the disjunction
- $\land$ is the conjunction
- $[(x_1 \lor x_2)$, $(\neg x_1 \lor x_3)$, $(\neg x_2 \lor \neg x_3)]$ are clauses

The formula is satisfiable if there is an assignment of truth values to the variables that makes the formula true. In this case, the formula is satisfiable if $x_1 = \text{True}$, $x_2 = \text{False}$, and $x_3 = \text{True}$.

There is the table with all possible assignments of truth values to the variables and the value of the formula:

| $x_1$ | $x_2$ | $x_3$ | $\phi ((x_1 \lor x_2) \land (\neg x_1 \lor x_3) \land (\neg x_2 \lor \neg x_3))$  |
|-------|-------|-------|---------|
| F     | F     | F     | F       |
| F     | F     | T     | F       |
| F     | T     | F     | T       |
| F     | T     | T     | F       |
| T     | F     | F     | F       |
| T     | F     | T     | T       |
| T     | T     | F     | F       |
| T     | T     | T     | F       |

To check truth of table, we can use following website: [Truth Table Generator](https://web.stanford.edu/class/cs103/tools/truth-table-tool/)


## How to solve SAT with Brute Force

To solve SAT we can use the **Brute Force** algorithm. It is a simple algorithm that checks all possible assignments of truth values to the variables in the formula.

1. Generate assignment of truth values to the variables.
2. For this assignment, check if the formula is true.
3. If the formula is true, return the assignment.
4. If the formula is false, go to step 1.

Algorithm stops when the formula is true for some assignment of truth values to the variables or all possible assignments are checked.

| Algorithm | Time Complexity | Space Complexity |
|-----------|-----------------|------------------|
| Brute Force | $O(2^n \cdot m)$ | $O(n)$ |

### Example

Let's consider the formula $(x_1 \lor x_2) \land (\neg x_1 \lor x_3) \land (\neg x_2 \lor \neg x_3)$.

We have 3 variables $x_1, x_2, x_3$. There are $2^3 = 8$ possible assignments of truth values to the variables:

1. generate first assignment: $x_1 = \text{F}$, $x_2 = \text{F}$, $x_3 = \text{F}$
2. check if the formula is true for this assignment: $\phi$ is false
3. ...
5. generate third assignment: $x_1 = \text{F}$, $x_2 = \text{T}$, $x_3 = \text{F}$
6. check if the formula is true for this assignment: $\phi$ is true
7. return the assignment: $x_1 = \text{F}$, $x_2 = \text{T}$, $x_3 = \text{F}$

### Implementation

In [55]:
def eval_formula(formula: list[list[int]], assignment: list[bool]) -> bool:
    print(f'assignment: {assignment}')
    for clause in formula:
        satisfied = False
        for var in clause:
            if var > 0:
                satisfied = satisfied or assignment[var-1]
            else:
                satisfied = satisfied or not assignment[-var-1]
        print(f'satisfied: {satisfied} with vars -> {clause}')
        if not satisfied:
            print(f'clause {clause} is not satisfied')
            return False
    return True

def brute_force_sat(formula: list[list[int]], num_vars: int) -> tuple[bool, list[bool]]:
    count = 1
    for bits in range(2**num_vars):
        assignment = [(bits >> i) & 1 == 1 for i in range(num_vars)]
        print(f'{count} -> assignment: {assignment}')
        count += 1
        if eval_formula(formula, assignment):
            return True, assignment
        print('---')
    return False, None

def main():
    formula = [[1, 2], [-1, 3], [-2, -3]]
    num_vars = 3
    sat, assignment = brute_force_sat(formula, num_vars)
    if sat:
        print(f'\nSAT: {sat} -> assignment: {assignment}')
    else:
        print("UNSAT")

if __name__ == "__main__":
    main()

1 -> assignment: [False, False, False]
assignment: [False, False, False]
satisfied: False with vars -> [1, 2]
clause [1, 2] is not satisfied
---
2 -> assignment: [True, False, False]
assignment: [True, False, False]
satisfied: True with vars -> [1, 2]
satisfied: False with vars -> [-1, 3]
clause [-1, 3] is not satisfied
---
3 -> assignment: [False, True, False]
assignment: [False, True, False]
satisfied: True with vars -> [1, 2]
satisfied: True with vars -> [-1, 3]
satisfied: True with vars -> [-2, -3]

SAT: True -> assignment: [False, True, False]


## SAT-Solver with DPLL (Davis-Putnam-Logemann-Loveland) Algorithm

The DPLL algorithm is a complete and efficient algorithm for solving the SAT problem. It is based on the backtracking search and unit propagation. The algorithm is named after its inventors: Martin Davis, George Logemann, Donald Loveland, and Hilary Putnam.

The DPLL algorithm works as follows:

1. **Unit Propagation**: If a clause is a unit clause (contains only one variable), the truth value of that variable is fixed to satisfy the clause.
2. **Pure Literal Elimination**: If there is a variable that appears only with one polarity in the formula, assign the truth value to the variable.
3. **Backtracking**: If the formula is unsatisfiable, backtrack to the last decision and change the assignment.

The DPLL algorithm stops when either all clauses are satisfied (solution found) or a contradiction is found (no solution under the current assignments).

| Algorithm | Time Complexity | Space Complexity |
|-----------|-----------------|------------------|
| DPLL | $O(2^n)$ | $O(n)$ |

To watch how DPLL algorithm works, you can use the following website: [DPLL SAT Solver](https://www.inf.ufpr.br/dpasqualin/d3-dpll/)

### Implementation

In [59]:
def dpll(clauses: list[list[int]], assignment: list[int] = [], count: int = 1) -> tuple[bool, list[int]]:
    # Base case: If there are no clauses left, all clauses have been satisfied.
    if not clauses:
        print(f'Step {count}: No clauses left -> Return: True, Final assignment: {assignment}')
        return True, assignment

    # Base case: If there is an empty clause, the problem is unsatisfiable with the current assignment.
    if any([not clause for clause in clauses]):
        print(f'Step {count}: Found empty clause -> Return: False, assignment: None')
        return False, None

    # Unit propagation: If a clause is a unit clause, its single variable must be set to satisfy the clause.
    for clause in clauses:
        if len(clause) == 1:
            unit = clause[0]
            # Propagate the unit throughout the formula and continue the search.
            # Exclude the unit and its negation and add the unit to the assignment.
            new_clauses = [ [var for var in c if var != -unit] for c in clauses if unit not in c]
            print(f'Step {count}: Found unit clause {unit} -> Propagate and continue with new clauses: {new_clauses}')
            return dpll(new_clauses, assignment + [unit], count + 1)

    # Choose the first variable in the first clause that is not a unit clause.
    variable = clauses[0][0]

    # Try assigning the variable to true and solve the remaining problem recursively.
    new_clauses = [ [var for var in c if var != -variable] for c in clauses if variable not in c]
    print(f'Step {count}: Try variable {variable} as True -> Continue with new clauses: {new_clauses}')
    sat, vals = dpll(new_clauses, assignment + [variable], count + 1)
    if sat:
        # If satisfiable with variable = true, return the solution.
        return sat, vals

    # If not satisfiable with variable = true, try variable = false.
    print(f'Step {count}: Try variable {variable} as False')
    return dpll([ [var for var in c if var != variable] for c in clauses if -variable not in c], assignment + [-variable])

def main():
    formula = [[1, 2], [-1, 3], [-2, -3]]
    sat, assignment = dpll(formula)
    if sat:
        print(f'\nSAT: {sat} -> assignment: {assignment}')
    else:
        print("UNSAT")

if __name__ == "__main__":
    main()

Step 1: Try variable 1 as True -> Continue with new clauses: [[3], [-2, -3]]
Step 2: Found unit clause 3 -> Propagate and continue with new clauses: [[-2]]
Step 3: Found unit clause -2 -> Propagate and continue with new clauses: []
Step 4: No clauses left -> Return: True, Final assignment: [1, 3, -2]

SAT: True -> assignment: [1, 3, -2]


### Comparison of Brute Force and DPLL

In [53]:
formulas = {
    3: [[1, 2], [-1, 3], [-2, -3]],
    4: [[1, 2, 3], [-1, 2, 4], [-2, -3, 4], [-1, -2, -3]],
    5: [[1, 2, 3], [-1, 2, 4], [-2, -3, 4], [-1, -2, -3], [1, 3, 5]],
    6: [[1, 2, 3], [-1, 2, 4], [-2, -3, 4], [-1, -2, -3], [1, 3, 5], [-1, -3, -5]],
    7: [[1, 2, 3], [-1, 2, 4], [-2, -3, 4], [-1, -2, -3], [1, 3, 5], [-1, -3, -5], [2, 4, 6]],
}

In [58]:
%%timeit
for num_vars, formula in formulas.items():
    dpll(formula)

36.1 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [54]:
%%timeit
for vars, formula in formulas.items():
    brute_force_sat(formula, vars)

38.1 µs ± 2.45 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
